<a href="https://colab.research.google.com/github/23520052/DoAnDuLieuLon/blob/main/4_Demo_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile app.py
import streamlit as st
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# --- CẤU HÌNH TRANG ---
st.set_page_config(page_title="ABSA Demo", page_icon="🍜", layout="centered")

st.title("🍜 Phân tích cảm xúc Nhà hàng")
st.caption("Dự án Big Data: PhoBERT + Back-Translation")

# --- LOAD MODEL TỪ DRIVE ---
@st.cache_resource
def load_model():
    # ĐƯỜNG DẪN TỚI THƯ MỤC CHỨA MODEL TRONG DRIVE
    model_path = '/content/drive/My Drive/ABSA_Project/Model_Final'

    # Load Tokenizer & Model (Tự động nhận diện safetensors hoặc bin)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    return tokenizer, model

try:
    with st.spinner('Đang khởi động AI... (Mất khoảng 30s)'):
        tokenizer, model = load_model()
    st.success("Hệ thống sẵn sàng!", icon="✅")
except Exception as e:
    st.error(f"Lỗi load model: {e}")
    st.stop()

# --- GIAO DIỆN ---
input_text = st.text_area("Nhập nhận xét:", height=100,
                          placeholder="Ví dụ: Đồ ăn ngon nhưng giá hơi chát.")

aspect = st.selectbox("Chọn khía cạnh:",
                     ["FOOD#QUALITY", "SERVICE#GENERAL",
                      "AMBIENCE#GENERAL", "RESTAURANT#PRICES"])

if st.button("Phân tích ngay", type="primary"):
    if not input_text:
        st.warning("Vui lòng nhập nội dung!")
    else:
        # Xử lý input theo chuẩn PhoBERT
        full_input = f"{aspect} {tokenizer.sep_token} {input_text}"
        inputs = tokenizer(full_input, return_tensors="pt", truncation=True, max_length=128)

        with torch.no_grad():
            outputs = model(**inputs)
            probs = F.softmax(outputs.logits, dim=1)
            confidence, pred = torch.max(probs, dim=1)

        # Hiển thị kết quả
        labels = {0: "TIÊU CỰC 😡", 1: "TRUNG LẬP 😐", 2: "TÍCH CỰC 😍"}
        colors = {0: "#ffcccc", 1: "#fff4cc", 2: "#ccffcc"}
        idx = pred.item()

        st.markdown(f"""
        <div style="padding: 20px; background-color: {colors[idx]}; border-radius: 10px; text-align: center;">
            <h2 style="color: #333; margin:0;">{labels[idx]}</h2>
            <p>Độ tin cậy: <b>{confidence.item():.1%}</b></p>
        </div>
        """, unsafe_allow_html=True)

In [ ]:
# Chạy ngầm server
!nohup streamlit run app.py &

# ⚠️ HƯỚNG DẪN:
# 1. Đăng ký tài khoản miễn phí tại: https://dashboard.ngrok.com/get-started/your-authtoken
# 2. Copy token của bạn và thay thế vào dòng bên dưới:

# Thay dòng chứa token thật bằng dòng này:
ngrok.set_auth_token("DÁN_MÃ_TOKEN_NGROK_CỦA_BẠN_VÀO_ĐÂY")

url = ngrok.connect(8501).public_url
print(f"🚀 LINK DEMO: {url}")